In [1]:
import sys
print(sys.executable)

/home/zhen/miniforge3/envs/spatial_Scanpy_Squidpy_GPU/bin/python


In [1]:
import scanpy as sc
import squidpy as sq
import matplotlib.pyplot as plt

/home/zhen/miniforge3/envs/spatial_Scanpy_Squidpy_GPU/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: The legacy Dask DataFrame implementation is deprecated and will be removed in a future version. Set the configuration option `dataframe.query-planning` to `True` or None to enable the new Dask Dataframe implementation and silence this warning.
  warnings.warn(


ModuleNotFoundError: No module named 'pkg_resources'

In [ ]:
# 1️⃣ 读取示例 Visium 空间转录组数据
adata = sq.datasets.visium_hne_adata()

# 2️⃣ 质量控制
sc.pp.calculate_qc_metrics(adata, inplace=True)

# 3️⃣ 归一化
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# 4️⃣ 高变基因
sc.pp.highly_variable_genes(adata, flavor="seurat", n_top_genes=2000)
adata = adata[:, adata.var.highly_variable]

# 5️⃣ PCA
sc.pp.scale(adata)
sc.tl.pca(adata)

# 6️⃣ 构建邻接图
sc.pp.neighbors(adata)

# 7️⃣ UMAP
sc.tl.umap(adata)

# 8️⃣ 聚类
sc.tl.leiden(adata, resolution=0.5)

# 9️⃣ 可视化 UMAP
sc.pl.umap(adata, color="leiden")

# 🔟 空间可视化
sc.pl.spatial(adata, color="leiden")

# 11️⃣ 空间邻域图
sq.gr.spatial_neighbors(adata)

# 12️⃣ 空间自相关分析（Moran’s I）
sq.gr.spatial_autocorr(adata, mode="moran")

# 13️⃣ 查看空间高变基因
print(adata.uns["moranI"].head())

# 14️⃣ 可视化空间高表达基因
top_gene = adata.uns["moranI"].index[0]
sc.pl.spatial(adata, color=top_gene)